# Учись Машина Учись / Learn Machine Learn

<img src="data/lml.png" width=200>

Онлайн-лекции Ильи С. Елисеева: применение методов машинного обучения в анализе данных.

- Канал в Telegram: https://t.me/learn_machine_learn
- YouTube: https://www.youtube.com/channel/UCCwDwKatNitBCVAJajremMQ
- VK: https://vk.com/learn_machine_learn
- GitHub: https://github.com/easyise/learn_machine_learn

---



# Лекция 2. Геоданные, пространственный анализ и избавление от выбросов


## 1. Получение данных из GPX файла и визуализация маршрута

Можно использовать ```pandas``` и ```gpxpy``` для извлечения данных из GPX файла и создания DataFrame.

In [ ]:
%pip install gpxpy geopy geopandas folium contextily folium ipywidgets

In [ ]:
import gpxpy
import geopy
import pandas as pd
import numpy as np

In [ ]:
gpx_file = "data/Afternoon_Backcountry_Ski.gpx"

with open(gpx_file, "r", encoding="utf-8") as f:
    gpx = gpxpy.parse(f)

gpx.tracks[0].segments[0].points[0]



In [ ]:
df_ski = pd.DataFrame([], columns=["time", "lat", "lon", "ele"])

ix = 0
for trk in gpx.tracks:
    for seg in trk.segments:
        for p in seg.points:
            df_ski.loc[ix] = {
                "time": pd.to_datetime(p.time),
                "lat": p.latitude,
                "lon": p.longitude,
                "ele": p.elevation if p.elevation is not None else pd.NA,
            }
            ix += 1

df_ski.set_index("time", inplace=True)
df_ski.sort_index(inplace=True) 
df_ski

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
plt.plot(df_ski["lon"], df_ski["lat"], "-r", lw=1)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("GPX track")
plt.axis("equal")
plt.grid()
plt.show()

Альтернативный вариант: использовать библиотеку ```geopandas``` для работы с геоданными и визуализации маршрута.

In [ ]:
import geopandas as gpd

gdf_ski = gpd.read_file(gpx_file, 
                        layer="track_points")
gdf_ski

In [ ]:
gdf_ski.plot(color="red", markersize=2, figsize=(8, 8))

In [ ]:
gdf_ski.dtypes

In [ ]:
# можно посчитать пройденный путь между точками с помощью метода distance
gdf_ski.distance(gdf_ski.shift(-1)).sum()

### Системы координат (CRS)

Для работы с **географическими координатами в градусах** в проекции WGS84 используется **EPSG:4326**:
- EPSG:4326 - WGS 84 - World Geodetic System 1984, используемая в GPS, координаты в градусах (широта, долгота)
- они находятся на поверхности эллипсоида, поэтому для измерения расстояний и площадей требуется преобразование в плоскую систему координат
- вычисления "в лоб" на географических координатах дают бессмысленные результаты.

Для работы с **плоскими координатами в метрах** часто используется проекция **EPSG:3857** (Web Mercator):
- EPSG:3857 - Web Mercator - широко используемая проекция для веб-карт, координаты в метрах (X, Y)
- позволяет выполнять точные измерения расстояний и площадей на плоскости
- однако искажает формы и размеры объектов, особенно на больших масштабах и вблизи полюсов.

Если нужно более точное измерение расстояний и площадей, особенно на больших территориях, рекомендуется использовать специализированные проекции, такие как UTM (Universal Transverse Mercator) или локальные проекции, адаптированные к конкретным регионам.

Для вычисления расстояний "по классике" можно использовать формулу гаверсинусов (```haversine```), но в данном случае мы будем использовать возможности библиотеки ```geopandas```.

Наиболее корректные результаты достигаются при использовании специализированных библиотек, таких как ```geopy``` или ```shapely```, которые учитывают кривизну Земли при вычислении расстояний между географическими координатами.

In [ ]:
# Система координат (CRS) нашего GeoDataFrame
gdf_ski.crs

In [ ]:
# Для вычисления расстояний в метрах "в общем виде" нужно преобразовать координаты в проекцию ESPG:3857
gdf_ski_3857 = gdf_ski[['time', 'geometry']].to_crs(epsg=3857)
gdf_ski_3857

In [ ]:
# так как дело происходит в Москве, можно использовать проекцию EPSG:32637
gdf_ski_32637 = gdf_ski[['time', 'geometry']].to_crs(epsg=32637)
gdf_ski_32637

In [ ]:
# можно заставить geopandas подобрать ESPG код автоматически
utm_crs = gdf_ski.estimate_utm_crs()
utm_crs

### Вычисление расстояний, скоростей перемещения и ускорений

Для вычисления расстояний между точками маршрута необходимо преобразовать координаты в проекцию с плоской системой координат (например, EPSG:3857). Затем можно использовать метод ```distance()``` для вычисления расстояний между последовательными точками.  

Все оставльное - по формулам, знакомым нам со школьных времен.

In [ ]:

# расчет расстояний в проекции EPSG:3857
gdf_ski_3857["prev_point"] = gdf_ski_3857.geometry.shift(1)
gdf_ski["dist_3857"] = gdf_ski_3857.geometry.distance(gdf_ski_3857["prev_point"])
display(gdf_ski[["time","geometry", "dist_3857"]])
# расчет расстояний в проекции EPSG:32637
gdf_ski_32637["prev_point"] = gdf_ski_32637.geometry.shift(1)
gdf_ski["dist_32637"] = gdf_ski_32637.geometry.distance(gdf_ski_32637["prev_point"])
display(gdf_ski[["time","geometry", "dist_32637"]])



In [ ]:
gdf_ski.geometry[0].x


In [ ]:
# расчет расстояний по формуле гаверсинусов
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # метры
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = phi2 - phi1
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2

    return 2 * R * math.asin(math.sqrt(a))

haversine(gdf_ski.geometry[0].y, gdf_ski.geometry[0].x, 
          gdf_ski.geometry[1].y, gdf_ski.geometry[1].x)

In [ ]:
gdf_ski['prev_point'] = gdf_ski.geometry.shift(1)
gdf_ski['dist_haversine'] = gdf_ski.apply(lambda row: haversine(
    row.geometry.y, row.geometry.x,
    row.prev_point.y if pd.notna(row.prev_point) else row.geometry.y,
    row.prev_point.x if pd.notna(row.prev_point) else row.geometry.x
), axis=1)
display(gdf_ski[["time","geometry", "dist_haversine"]])

In [ ]:
# расчет расстояний при помощи pyproj и геопанды
from pyproj import Geod
geod = Geod(ellps="WGS84")
gdf_ski['prev_point'] = gdf_ski.geometry.shift(1)
def geod_distance(row):
    if pd.isna(row.prev_point):
        return 0.0
    lon1, lat1 = row.geometry.x, row.geometry.y
    lon2, lat2 = row.prev_point.x, row.prev_point.y
    az12, az21, dist = geod.inv(lon1, lat1, lon2, lat2)
    return dist

gdf_ski['dist_geod'] = gdf_ski.apply(geod_distance, axis=1)
display(gdf_ski[["time","geometry", "dist_geod"]])

In [ ]:
# Сравнение результатов
display(gdf_ski[["time","geometry", "dist_3857", "dist_32637", "dist_haversine", "dist_geod"]])
print("Суммарные расстояния:")
print(gdf_ski[["dist_3857", "dist_32637", "dist_haversine", "dist_geod"]].sum())

### Визуализация маршрута на карте

Простая и быстрая визуализация маршрута может быть выполнена с помощью метода ```plot()``` библиотеки ```geopandas```, который позволяет отобразить геометрические объекты на карте, которая будет загружена из интернета с помощью библиотеки ```contextily```.

In [ ]:
# Визуализация маршрута на карте - по умолчанию визуализация точек
import contextily as cx 

ax = gdf_ski.to_crs(epsg=3857).plot(figsize=(8, 8), linewidth=0.5)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)  # тянет тайлы из интернета
ax.set_axis_off()
plt.show()

In [ ]:
# если мы хотим связности - надо создать сегменты между точками
from shapely.geometry import LineString

def get_with_segments(gdf):
    gdf['prev_point'] = gdf.geometry.shift(1)
    gdf['segment'] = gdf.apply(lambda row: LineString([row.prev_point, row.geometry]) 
                               if pd.notna(row.prev_point) else None, axis=1)
    
    gdf_ret = gpd.GeoDataFrame(
        gdf, geometry='segment', crs=gdf.crs
    )

    return gdf_ret

gdf_segments = get_with_segments(gdf_ski)

ax = gdf_segments[['time', 'geometry', 'segment']] \
    .to_crs(epsg=3857) \
    .dropna() \
    .plot(figsize=(8, 8), linewidth=0.5, color='blue')
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_axis_off()
plt.show()

Чтобы раскрасить маршрут в зависимости от скорости, можно использовать параметр ```column``` в методе ```plot()```, передав ему наименование признака, который мы хотим визуализировать. 

In [ ]:
# функция расчета расстояний, скоростей и ускорений
def recalculate_metrics(gdf_):
    # Calculate distances in meters by haversine formula
    gdf_['prev_point'] = gdf_.geometry.shift(1)
    gdf_['dist'] = gdf_.apply(lambda row: haversine(
        row.geometry.y, row.geometry.x,
        row.prev_point.y if pd.notna(row.prev_point) else row.geometry.y,
        row.prev_point.x if pd.notna(row.prev_point) else row.geometry.x
    ), axis=1)

    # Calculate time differences in seconds
    gdf_['timedelta_s'] = gdf_['time'].diff().dt.total_seconds()
    gdf_['timedelta_s'] = gdf_['timedelta_s'].fillna(0)

    # Calculate speed in m/s
    gdf_['speed_kmh'] = gdf_['dist'] / gdf_['timedelta_s'] * 3.6
    gdf_['speed_kmh'] = gdf_['speed_kmh'].fillna(0)

    # Calculate acceleration in m/s² (change in speed over time)
    # gdf_['ds'] = gdf_['speed_kmh'].diff()
    gdf_['acc_m_per_s2'] = (gdf_['speed_kmh'] / 3.6).diff() / gdf_['timedelta_s']
    gdf_['acc_g'] = gdf_['acc_m_per_s2'] / 9.81

    gdf_[['acc_m_per_s2', 'acc_g']] = gdf_[['acc_m_per_s2', 'acc_g']].fillna(0)
    
    return gdf_

gdf_ = recalculate_metrics(gdf_ski)
gdf_[['time', 'ele', 'dist', 'timedelta_s', 'speed_kmh', 'acc_m_per_s2', 'acc_g', 'prev_point', 'geometry']].head(20)

In [ ]:
ax = get_with_segments(gdf_)[['time', 'segment', 'speed_kmh']] \
    .to_crs(epsg=3857) \
    .plot(
        column="speed_kmh",
        cmap="turbo",
        vmin=0,
        vmax=30,
        legend=True,
        figsize=(8, 8),
)

ax.set_title("GPX track colored by speed (km/h)")
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_axis_off()
plt.show()

Для визуализации маршрута на карте лучше использовать библиотеку ```folium```, которая позволяет создавать интерактивные карты с возможностью добавления маркеров, линий и других элементов.

In [ ]:
latlon = locations=[(row.geometry.y, row.geometry.x) for idx, row in gdf_.iterrows()]
latlon

In [ ]:
# Визуализация маршрута на карте с помощью folium
import folium

m = folium.Map(location=[gdf_.geometry.y.mean(), gdf_.geometry.x.mean()], zoom_start=13)
folium.PolyLine(latlon,
                color="red", 
                weight=2.5, 
                opacity=1)\
    .add_to(m)
m

In [ ]:
import matplotlib
import matplotlib.colors as mcolors
import numpy as np

latlon = [(row.geometry.y, row.geometry.x) for idx, row in gdf_.iterrows()]
speeds = gdf_['speed_kmh'].values

vmin = 0
vmax = 60

# Нормализуем для colormap
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = matplotlib.colormaps['turbo']
colors = [mcolors.to_hex(cmap(norm(s))) for s in speeds]

# Создаём карту
m = folium.Map(location=[gdf_.geometry.y.mean(), gdf_.geometry.x.mean()], zoom_start=13)

# Рисуем сегменты разными цветами
for i in range(1, len(latlon)):
    folium.PolyLine(
        locations=[latlon[i-1], latlon[i]],
        color=colors[i],
        weight=4,
        opacity=0.8
    ).add_to(m)

# Генерируем градиент автоматически
gradient_steps = 10
gradient_html = ''.join([
    f'<span style="display:inline-block; width:20px; height:20px; background-color:{mcolors.to_hex(cmap(i/gradient_steps))};"></span>'
    for i in range(gradient_steps + 1)
])

legend_html = f'''
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 180px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <p style="margin: 0 0 5px 0;"><b>Скорость (км/ч)</b></p>
     <p style="margin: 5px 0;">Min: {vmin:.1f}</p>
     <p style="margin: 5px 0;">Max: {vmax:.1f}</p>
     <div style="margin-top: 10px;">
         {gradient_html}
     </div>
     <p style="margin: 5px 0 0 0; font-size: 12px; text-align: center;">
         {vmin:.1f} → {vmax:.1f}
     </p>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))


m

In [ ]:
from folium import plugins

latlon = locations=[(row.geometry.y, row.geometry.x) for idx, row in gdf_ski.iterrows()]

m = folium.Map(location=[gdf_ski.geometry.y.mean(), gdf_ski.geometry.x.mean()], zoom_start=13)

# весь трек, статично
folium.PolyLine(latlon, weight=4, opacity=0.35).add_to(m)

# бегущая линия
plugins.AntPath(
    locations=latlon,
    delay=350,          # меньше = быстрее
    weight=6,
    opacity=0.9,
    dash_array=[10, 20],
    pulse_color="#ffffff",  # “ядро”
    color="#ff9900"         # “свечение”
).add_to(m)

m

In [ ]:
import pandas as pd

# 1) подготовка: время -> datetime, сортировка
df = gdf_ski[["time", "geometry"]].dropna().copy()
df["time"] = pd.to_datetime(df["time"], errors="coerce", utc=True)
df = df.dropna(subset=["time"]).sort_values("time").reset_index(drop=True)

# 2) извлекаем lon/lat из Point
lons = df["geometry"].apply(lambda p: float(p.x))
lats = df["geometry"].apply(lambda p: float(p.y))

# 3) собираем features: сегменты LineString между соседними точками
features = []
for i in range(1, len(df)):
    prev = (lons.iloc[i-1], lats.iloc[i-1])  # (lon, lat) для GeoJSON
    cur  = (lons.iloc[i],   lats.iloc[i])

    features.append({
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [prev, cur],
        },
        "properties": {
            "time": df["time"].iloc[i].isoformat(),
            "style": {"color": "#ff9900", "weight": 5, "opacity": 0.9},
        }
    })

# 4) итоговая структура для TimestampedGeoJson
geojson = {"type": "FeatureCollection", "features": features}

geojson  # можно посмотреть, что получилось

In [ ]:
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[lats.iloc[0], lons.iloc[0]], zoom_start=13, tiles="OpenStreetMap")

# весь трек, статично
folium.PolyLine(latlon, weight=4, opacity=0.35).add_to(m)

# маркеры начала и конца
folium.Marker(latlon[0], popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(latlon[-1], popup="End", icon=folium.Icon(color="red")).add_to(m)

TimestampedGeoJson(
    geojson,
    period="PT1M",  # интервал между кадрами (1 минута)
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=10,
    loop_button=True,
    time_slider_drag_update=True,
).add_to(m)

m

In [ ]:
from folium.plugins import TimestampedGeoJson

# точки для анимации (круг)
point_features = []
for i in range(len(df)):
    point_features.append({
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lons.iloc[i], lats.iloc[i]]},
        "properties": {
            "time": df["time"].iloc[i].isoformat(),
            "icon": "circle",
            "iconstyle": {
                "fillColor": "#ff9900",
                "fillOpacity": 0.9,
                "stroke": False,
                "radius": 6,
            },
        },
    })

geojson_points = {"type": "FeatureCollection", "features": point_features}

m = folium.Map(location=[lats.iloc[0], lons.iloc[0]], zoom_start=13, tiles="OpenStreetMap")

# весь трек, статично
folium.PolyLine(latlon, weight=4, opacity=0.35).add_to(m)

# маркеры начала и конца
folium.Marker(latlon[0], popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(latlon[-1], popup="End", icon=folium.Icon(color="red")).add_to(m)

# ползунок с движущейся точкой
TimestampedGeoJson(
    geojson_points,
    period="PT1M",
    add_last_point=False,
    auto_play=True,
    loop=True,
    max_speed=10,
    loop_button=True,
    time_slider_drag_update=True,
).add_to(m)

m